In [1]:
import numpy as np
import ast
from copy import deepcopy

# Part 1

In [2]:
def depth(snail):
    #determine depth of snail
    if isinstance(snail, list):
        return 1 + max(depth(item) for item in snail)
    else:
        return 0
    
def flatten(snail):
    #function to flatten snail into slug
    slug = []
    for i in range(0, 2):
        if isinstance(snail[i], list):
            slug += flatten(snail[i])
        else:
            slug.append(snail[i])
    return slug

def depth_idx(snail):
    #find index of first deepest point
    idx = -1
    for i in range(0, 2):
        idx += 1
        if isinstance(snail[i], list):
            for j in range(0, 2):
                if j == 1:
                    idx += 1
                if isinstance(snail[i][j], list):
                    for k in range(0, 2):
                        if k == 1:
                            idx += 1
                        if isinstance(snail[i][j][k], list):
                            for l in range(0, 2):
                                if l == 1:
                                    idx += 1
                                if isinstance(snail[i][j][k][l], list):
                                    return idx, [i,j,k,l]
                                
def curl_slug(snail, slug, slug_idx=None):
    #function to curl slug back up into snail
    idx = 0
    for i in range(0, 2):
        if i == 1:
            idx += 1
        if isinstance(snail[i], list):
            for j in range(0, 2):
                if j == 1:
                    idx += 1
                if isinstance(snail[i][j], list):
                    for k in range(0, 2):
                        if k == 1:
                            idx += 1
                        if isinstance(snail[i][j][k], list):
                            for l in range(0, 2):
                                if l == 1:
                                    idx += 1
                                if isinstance(snail[i][j][k][l], list) and slug_idx is not None and idx != slug_idx:
                                    for m in range(0, 2):
                                        if m == 1:
                                            idx += 1
                                        snail[i][j][k][l][m] = slug[idx]
                                        if idx == len(slug)-1:
                                            return snail
                                else:
                                    snail[i][j][k][l] = slug[idx]
                        else:
                            snail[i][j][k] = slug[idx]
                else:
                    snail[i][j] = slug[idx]
        else:
            snail[i] = slug[idx]
    return snail
        
def explode(snail):
    #explode deep numbers
    slug = flatten(snail)
    idx = depth_idx(snail)
    slug_idx, snail_idx = idx
    i,j,k,l = snail_idx
    
    if slug_idx == 0:
        slug[0] = 0
        slug[2] += slug[1]
        del slug[1]
        
    elif slug_idx == len(slug)-2:
        slug[-1] = 0
        slug[-3] += slug[-2]
        del slug[-2]
        
    else:
        slug[slug_idx-1] += slug[slug_idx]
        del slug[slug_idx]
        slug[slug_idx+1] += slug[slug_idx]
        slug[slug_idx] = 0
        
    #print(slug)
    snail = curl_slug(snail, slug, slug_idx)
        
    return snail

def split(snail, tens):
    #split values > 10
    slug = flatten(snail)
    val = slug[tens[0]]
    slug[tens[0]] = [int(np.floor(val/2)), int(np.ceil(val/2))]
    snail = curl_slug(snail, slug)
    return snail

def add_numbers(numbers):
    #add snail numbers
    snail = numbers[0]
    for i in range(1, len(numbers)):
        snail = [snail, numbers[i]]
        while True:
            while depth(snail) == 5:
                snail = explode(snail)
            tens = np.where(np.array(flatten(snail)) > 9)[0]
            if len(tens) > 0:
                snail = split(snail, tens)

            tens = np.where(np.array(flatten(snail)) > 9)[0]
            dpth = depth(snail)
            if len(tens) == 0 and dpth < 5:
                break
    return snail
    
def magnitude(snail):
    #get magnitude of snail numbers
    for i in range(0, 2):
        if isinstance(snail[i], list):
            snail[i] = magnitude(snail[i])
        elif i == 1:
            return (3*snail[0]) + (2*snail[1])
    return (3*snail[0]) + (2*snail[1])
    
test1 = [[1,1],[2,2],[3,3],[4,4]]
test2 = [[1,1],[2,2],[3,3],[4,4],[5,5]]
test3 = [[1,1],[2,2],[3,3],[4,4],[5,5],[6,6]]
test4 = [ [[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]],
          [7,[[[3,7],[4,3]],[[6,3],[8,8]]]],
          [[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]],
          [[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]],
          [7,[5,[[3,8],[1,4]]]],
          [[2,[2,2]],[8,[8,1]]],
          [2,9],
          [1,[[[9,3],9],[[9,0],[0,7]]]],
          [[[5,[7,4]],7],1],
          [[[[4,2],2],6],[8,7]] ]
add_numbers(test1)
add_numbers(test2)
add_numbers(test3)
add_numbers(test4)

test5 = [ [[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],
          [[[5,[2,8]],4],[5,[[9,9],0]]],
          [6,[[[6,2],[5,6]],[[7,6],[4,7]]]],
          [[[6,[0,7]],[0,9]],[4,[9,[9,0]]]],
          [[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]],
          [[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]],
          [[[[5,4],[7,7]],8],[[8,3],8]],
          [[9,3],[[9,9],[6,[4,9]]]],
          [[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]],
          [[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]] ]
print(magnitude(add_numbers(test5)))

4140


In [3]:
def process_input():
    inpt = np.genfromtxt('day18_input.txt', dtype=str, delimiter='\n')
    numbers = []
    for line in inpt:
        numbers.append(ast.literal_eval(line))
    return numbers

print('Part 1 Result:', magnitude(add_numbers(process_input())))

Part 1 Result: 4433


# Part 2

In [4]:
def get_max_sum(numbers):
    maxi = 0
    for i in range(0, len(numbers)):
        for j in range(i+1, len(numbers)):
            val = magnitude(add_numbers([deepcopy(numbers[i]),deepcopy(numbers[j])]))
            if val > maxi:
                maxi = val
            #addition is not commutative
            val = magnitude(add_numbers([deepcopy(numbers[j]),deepcopy(numbers[i])]))
            if val > maxi:
                maxi = val
    return maxi

test6 = [ [[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],
          [[[5,[2,8]],4],[5,[[9,9],0]]],
          [6,[[[6,2],[5,6]],[[7,6],[4,7]]]],
          [[[6,[0,7]],[0,9]],[4,[9,[9,0]]]],
          [[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]],
          [[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]],
          [[[[5,4],[7,7]],8],[[8,3],8]],
          [[9,3],[[9,9],[6,[4,9]]]],
          [[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]],
          [[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]] ]
print(get_max_sum(test6))

3993


In [5]:
print('Part 2 Result:', get_max_sum(process_input()))

Part 2 Result: 4559
